In [5]:
import pandas as pd
import plotly.express as px
import numpy as np

# Load the data
df = pd.read_csv("microplastics_daily_2.5deg_2019.csv")
df["date_label"] = pd.to_datetime(df["date"]).dt.strftime("%b %d\n%Y")

# Adjusted VIIRS-inspired color scale with more contrast
viirs_scale = [
    [0.0, "#000000"],   # black (no concentration)
    [0.1, "#000000"],   # black for better contrast
    [0.3, "#000000"],   # black to emphasize hotspots
    [0.5, "#ffd92f"],   # yellow
    [0.75, "#fc4e2a"],  # orange-red
    [1.0, "#bd0026"],   # deep red
]

# Set range for color scale
vmin = df["mp_concentration_mean"].min()
vmax = df["mp_concentration_mean"].max()



# Create the VIIRS-style visualization - explicitly use dates for animation
fig = px.scatter_geo(
    df,
    lat="lat_bin",
    lon="lon_bin",
    color="mp_concentration_mean",
    size_max=3,
    animation_frame="date_label", 
    title="Global Microplastic Concentration (2019)",
    color_continuous_scale=viirs_scale,
    projection="natural earth",
    range_color=[vmin, vmax],
    hover_data={
        "lat_bin": False,
        "lon_bin": False,
        "ocean_name": True,
        "mp_concentration_mean": True
    },
    opacity=0.8
)

# Fix the map center and zoom level for stability in animation
fig.update_geos(
    showland=True,
    landcolor="#1f3b5c",      # land color
    showocean=True,
    oceancolor="#000000",     # black ocean
    showcoastlines=True,
    coastlinecolor="#1f3b5c", # subtle blue coastline
    showcountries=False,
    showlakes=False,
    bgcolor="#000000",        # black background
    projection_scale=1,       # Fixed scale
    center=dict(lat=10, lon=0), # Fixed center
    lataxis=dict(range=[-90, 90], showgrid=False), # Fixed latitude range
    lonaxis=dict(range=[-180, 180], showgrid=False) # Fixed longitude range
)

# Stabilize layout to prevent movement during animation
fig.update_layout(
    coloraxis_colorbar=dict(
        tickfont=dict(color='white'),
        title=dict(
            text="Microplastics Concentration",
            font=dict(color='white', size=20, family='Arial Black')
        ),
        tickvals=[vmin, vmax],
        ticktext=["Lower", "Higher"],
        len=0.7,
        x=1.02,              # Fixed position
        y=0.5,               # Fixed position
        yanchor="middle"     # Fixed anchor point
    ),
    font=dict(size=16, color='white', family='Arial Black'),
    title=dict(
        text="Global Microplastic Concentration (2019)",
        font=dict(size=18, color='white', family='Arial Black'),
        x=0.5,   # Center title
        y=0.95   # Fixed position
    ),
    margin=dict(l=0, r=0, t=50, b=0, pad=0), # Fixed margins, no padding
    paper_bgcolor="black",
    plot_bgcolor="black",
    geo=dict(
        showframe=False,
        projection_type="natural earth",
        bgcolor="black",
        scope="world",   # Fixed scope
        fitbounds=False  # Don't auto-adjust bounds
    ),
    height=600,
    width=1200
)

# Update marker settings
fig.update_traces(
    marker=dict(
        line=dict(width=0),    # No marker border
        opacity=0.8,           # Slight transparency
        sizemin=1
    )
)

# Set animation options for smooth transitions
fig.layout.updatemenus = [{
    "buttons": [
        {
            "args": [None, {"frame": {"duration": 500, "redraw": True},
                         "fromcurrent": True,
                         "transition": {"duration": 300, "easing": "cubic-in-out"}}],
            "label": "Play",
            "method": "animate"
        },
        {
            "args": [[None], {"frame": {"duration": 0, "redraw": False},
                           "mode": "immediate",
                           "transition": {"duration": 0}}],
            "label": "Pause",
            "method": "animate"
        }
    ],
    "direction": "left",
    "pad": {"r": 10, "t": 10},
    "showactive": False,
    "type": "buttons",
    "x": 0.1,
    "y": 0,
    "xanchor": "right",
    "yanchor": "top"
}]

# Fix for the slider
if hasattr(fig.layout, 'sliders') and fig.layout.sliders:
    fig.layout.sliders[0].pad = dict(r=10, t=10, b=10)
    fig.layout.sliders[0].x = 0.1
    fig.layout.sliders[0].y = 0
    fig.layout.sliders[0].currentvalue = dict(
        font=dict(color="white"),
        prefix="Date: ",
        visible=True,
        xanchor="right"
    )

# Save to HTML file
fig.write_html("animated_map_nasagibs_style.html", include_plotlyjs="cdn")

# Display the figure
#fig.show()